In [1]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv", sep=",")

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))

In [3]:
def preprocess_features(california_housing_dataframe):
  """Prepares input features from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
  selected_features = california_housing_dataframe[
    ["latitude",
     "longitude",
     "housing_median_age",
     "total_rooms",
     "total_bedrooms",
     "population",
     "households",
     "median_income"]]
  processed_features = selected_features.copy()
  # Create a synthetic feature.
  processed_features["rooms_per_person"] = (
    california_housing_dataframe["total_rooms"] /
    california_housing_dataframe["population"])
  return processed_features

def preprocess_targets(california_housing_dataframe):
  """Prepares target features (i.e., labels) from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  # Scale the target to be in units of thousands of dollars.
  output_targets["median_house_value"] = (
    california_housing_dataframe["median_house_value"] / 1000.0)
  return output_targets

In [5]:
# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

Training examples summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0
mean,35.6,-119.5,28.6,2644.9,539.4,1429.1,501.3,3.9,2.0
std,2.1,2.0,12.6,2202.4,421.0,1124.4,385.1,1.9,1.3
min,32.5,-124.3,1.0,2.0,1.0,3.0,1.0,0.5,0.0
25%,33.9,-121.8,18.0,1465.0,297.0,791.8,282.0,2.6,1.5
50%,34.2,-118.5,29.0,2141.0,437.0,1171.0,410.0,3.5,1.9
75%,37.7,-118.0,37.0,3157.0,649.0,1722.0,606.0,4.7,2.3
max,42.0,-114.3,52.0,37937.0,6445.0,28566.0,6082.0,15.0,55.2


Validation examples summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,35.7,-119.6,28.5,2640.7,539.4,1430.8,501.1,3.9,2.0
std,2.1,2.0,12.6,2125.3,422.8,1202.4,383.3,1.9,0.9
min,32.5,-124.3,1.0,24.0,3.0,13.0,2.0,0.5,0.1
25%,33.9,-121.8,18.0,1453.8,295.0,783.0,280.8,2.6,1.5
50%,34.3,-118.5,29.0,2104.5,427.5,1156.0,405.0,3.6,1.9
75%,37.7,-118.0,37.0,3136.0,648.0,1718.0,604.0,4.8,2.3
max,42.0,-114.5,52.0,27700.0,4819.0,35682.0,4769.0,15.0,18.3


Training targets summary:


,median_house_value
count,12000.0
mean,207.0
std,116.2
min,15.0
25%,118.9
50%,179.5
75%,264.1
max,500.0


Validation targets summary:


,median_house_value
count,5000.0
mean,208.0
std,115.5
min,17.5
25%,121.4
50%,181.4
75%,268.0
max,500.0


In [10]:
def construct_feature_columns(input_features):
    """Construct the TensorFlow Feature Columns.

    Args:
    input_features: The names of the numerical input features to use.
    Returns:
    A set of feature columns
    """
    return set([tf.feature_column.numeric_column(feature) for feature in input_features])

In [11]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
   
    features = {key:np.array(value) for key,value in dict(features).items()}
    
    ds = Dataset.from_tensor_slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
    
    features, labels = ds.make_one_shot_iterator().get_next()
    
    return features, labels

In [12]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    feature_columns,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    """Trains a linear regression model.

    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.

    Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    feature_columns: A `set` specifying the input feature columns to use.
    training_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for validation.

    Returns:
    A `LinearRegressor` object trained on the training data.
    """

    periods = 10
    steps_per_period = steps / periods

    # Create a linear regressor object.
    my_optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["median_house_value"], 
                                          batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["median_house_value"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["median_house_value"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("RMSE (on training data):")
    training_rmse = []
    validation_rmse = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        linear_regressor.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        # Take a break and compute predictions.
        training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([item['predictions'][0] for item in training_predictions])
        validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])

        # Compute training and validation loss.
        training_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(training_predictions, training_targets))
        validation_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(validation_predictions, validation_targets))
        # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
        # Add the loss metrics from this period to our list.
        training_rmse.append(training_root_mean_squared_error)
        validation_rmse.append(validation_root_mean_squared_error)
    print("Model training finished.")


    # Output a graph of loss metrics over periods.
    plt.ylabel("RMSE")
    plt.xlabel("Periods")
    plt.title("Root Mean Squared Error vs. Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label="training")
    plt.plot(validation_rmse, label="validation")
    plt.legend()

    return linear_regressor

In [13]:
_ = train_model(
    learning_rate=1.0,
    steps=500,
    batch_size=100,
    feature_columns=construct_feature_columns(training_examples),
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Training model...
RMSE (on training data):
  period 00 : 194.53
  period 01 : 161.01
  period 02 : 112.92
  period 03 : 110.35
  period 04 : 110.15
  period 05 : 111.64
  period 06 : 117.79
  period 07 : 121.26
  period 08 : 133.10
  period 09 : 118.02
Model training finished.


In [39]:
def get_quantile_based_boundaries(feature_values, num_buckets):
    boundaries = np.arange(1.0, num_buckets)/num_buckets
    quantiles = feature_values.quantile(boundaries)
    return [quantiles[b] for b in quantiles.keys()]

In [34]:
get_quantile_based_boundaries(california_housing_dataframe["longitude"], 10)

[-122.28,
 -121.98,
 -121.36000000000001,
 -119.87,
 -118.49,
 -118.3,
 -118.12,
 -117.88,
 -117.24]

In [92]:
california_housing_dataframe.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')

In [103]:
def construct_feature_columns(training_examples):
    households = tf.feature_column.numeric_column('households')
    longitude = tf.feature_column.numeric_column("longitude")
    latitude = tf.feature_column.numeric_column("latitude")
    housing_median_age = tf.feature_column.numeric_column("housing_median_age")
    median_income = tf.feature_column.numeric_column("median_income")
    rooms_per_person = tf.feature_column.numeric_column("rooms_per_person")
  

    bucketized_households = tf.feature_column.bucketized_column(
        households,
        boundaries=get_quantile_based_boundaries(
            training_examples['households'],7))
    
    # Divide longitude into 10 buckets.
    bucketized_longitude = tf.feature_column.bucketized_column(
    longitude, boundaries=get_quantile_based_boundaries(
      training_examples["longitude"], 10))
    
    bucketized_latitude = tf.feature_column.bucketized_column(
        latitude,
        boundaries=get_quantile_based_boundaries(
            training_examples['latitude'],7))

    bucketized_housing_median_age = tf.feature_column.bucketized_column(
        housing_median_age,
        boundaries=get_quantile_based_boundaries(
            training_examples['housing_median_age'],10))
    
    bucketized_median_income = tf.feature_column.bucketized_column(
        median_income,
        boundaries=get_quantile_based_boundaries(
            training_examples['median_income'],10))
    
    bucketized_rooms_per_person = tf.feature_column.bucketized_column(
        rooms_per_person,
        boundaries=get_quantile_based_boundaries(
            training_examples['rooms_per_person'],10))
    
    long_x_lat = tf.feature_column.crossed_column([bucketized_latitude, bucketized_longitude], 110)
    long_x_lat_x_hholds = tf.feature_column.crossed_column([bucketized_longitude,
                                                           bucketized_latitude, 
                                                            bucketized_households],
                                                          1000)
    
    feature_columns = set([
        bucketized_households,
        bucketized_longitude,
        bucketized_latitude,
        bucketized_housing_median_age,
        bucketized_median_income,
        bucketized_rooms_per_person,
        long_x_lat_x_hholds
#         long_x_lat
    ])
    return feature_columns

In [104]:
_ = train_model(
    learning_rate=1,
    steps=500,
    batch_size=100,
    feature_columns=construct_feature_columns(training_examples),
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Training model...
RMSE (on training data):
  period 00 : 167.46
  period 01 : 139.91
  period 02 : 122.84
  period 03 : 111.47
  period 04 : 103.35
  period 05 : 97.53
  period 06 : 93.02
  period 07 : 89.41
  period 08 : 86.52
  period 09 : 84.07
Model training finished.


In [74]:
ftrs = construct_feature_columns(training_examples.head(10))

In [89]:
cc = ftrs.pop()

In [90]:
cc

_CrossedColumn(keys=(_BucketizedColumn(source_column=_NumericColumn(key='latitude', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(33.832857142857144, 34.10142857142857, 34.144285714285715, 34.69857142857143, 37.88285714285714, 37.971428571428575)), _BucketizedColumn(source_column=_NumericColumn(key='longitude', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(-122.26999999999998, -122.262, -122.25300000000001, -119.862, -118.215, -118.156, -118.07500000000002, -117.804, -117.405))), hash_bucket_size=110, hash_key=None)

In [91]:
cc.

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().